# Labo 3 - ARN

In [ ]:
import pandas as pd

mice_eeg = pd.read_csv('./pw3_data/EEG_mouse_data_1.csv')
mice_eeg2 = pd.read_csv('./pw3_data/EEG_mouse_data_2.csv')
df = pd.concat([mice_eeg, mice_eeg2], ignore_index=True)

mice_eeg3 = pd.read_csv('./pw3_data/EEG_mouse_data_test.csv')
df

## 1. Introduction

We take the first 25 features as they are contain the biggest occurences count.

In [ ]:
input_data = df.iloc[:, 1:26].to_numpy()
input_data

Normalisation


In [ ]:
# Shuffle
# df.sample(frac=1) #TODO: obsolete because of KFold ?

# Cut in 3 folds (1/3 for test and 2/3 for training)
# shuffle d'abord avant de découper
#  normaliser tout fit
# partager

import keras
from keras import layers
from sklearn.model_selection import KFold
import matplotlib.pyplot as pl


pl.clf()

keras.utils.set_random_seed(123)
kf = KFold(n_splits=3, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(df)):
    print(str(i) + ' ' + str(train_index) + ' ' + str(test_index))
# See training 2 cells below

In [ ]:
# Configure and setup our MLP
# Source: From the notebook 5. See 'Changed: '
def create_model():
  mlp = keras.Sequential([
      layers.Input(shape=(25,)), # Changed: shape=(25,) instead of 2
      layers.Dense(4, activation="tanh"), # Try different numbers of hidden neurons here (e.g. 2, 4, 8, 32, 128)
      # Changed: todo try to change the amount of layers and perceptrons per layer
      layers.Dense(1, activation="tanh"),   # next 
  ])

  # Experiment with hyperparameters here:
  # momentum: [0, 0.8, 0.9, 0.99]
  # learning_rate: [0.1, 0.01, 0.001, 0.0001]
  mlp.compile(
      optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.99),
      loss="mse",
  )

  return mlp

mlp = create_model()
mlp.summary()


In [ ]:
df

In [ ]:
# Create dataframe with first column "state" and replace n with 1 and other values with -1, 
# convert it as numpy array to access rows and not columns
output_data = df.iloc[:,0].map(lambda e: 1 if e == 'n' else -1).to_numpy()
output_data

In [ ]:
# Run training on 3 folds
history_list = []
trained_mlp = []

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

print(output_data.shape)
for i, (train_index, test_index) in enumerate(kf.split(input_data)):
  print(str(i) + ' ' + str(train_index) + ' ' + str(test_index))
  
  # We need to create a new model everytime otherwise fit will continue previous training
  mlp = create_model()
  
  train_input = scaler.fit_transform(input_data[train_index])
  test_input = scaler.fit_transform(input_data[test_index])
  
  history = mlp.fit(
      x=train_input, 
      y=output_data[train_index],
      validation_data=(test_input, output_data[test_index]),
      epochs=3
  )

  history_list.append(history)
  trained_mlp.append(mlp)


In [ ]:
trained_mlp[0]

## Plots of training and validation loss

In [ ]:
import numpy as np

train_losses = np.array([history.history['loss'] for history in history_list])
val_losses = np.array([history.history['val_loss'] for history in history_list])

# Calculate mean and standard deviation for training and validation losses
mean_train_loss = np.mean(train_losses, axis=0)
std_train_loss = np.std(train_losses, axis=0)
mean_val_loss = np.mean(val_losses, axis=0)
std_val_loss = np.std(val_losses, axis=0)

# Plot mean and standard deviation for training loss
pl.plot(mean_train_loss, label='Training Loss (Mean)')
pl.fill_between(range(len(mean_train_loss)), mean_train_loss - std_train_loss, mean_train_loss + std_train_loss, alpha=0.3, label='Training Loss (Std)')

# Plot mean and standard deviation for validation loss
pl.plot(mean_val_loss, label='Validation Loss (Mean)')
pl.fill_between(range(len(mean_val_loss)), mean_val_loss - std_val_loss, mean_val_loss + std_val_loss, alpha=0.3, label='Validation Loss (Std)')

# Add labels and legend
pl.xlabel('Epochs')
pl.ylabel('Loss')
pl.legend()

# Display the plot
pl.show()

## test

In [ ]:
for idx, model in enumerate(trained_mlp):
    # Plot model's output in the feature space
    input_x = np.arange(-1.2, 1.2, 0.1)
    input_y = np.arange(-1.2, 1.2, 0.1)
    input_x_matrix, input_y_matrix = np.meshgrid(input_x, input_y)
    inputs_xy = np.concatenate((input_x_matrix.flatten()[:,np.newaxis], input_y_matrix.flatten()[:,np.newaxis]), axis=1)

    output_values = model(inputs_xy)
    output_matrix = np.reshape(output_values, input_x_matrix.shape)

    pl.figure(figsize=(8,8))
    img = pl.imshow(np.flipud(output_matrix), interpolation='None', extent=(-1.2,1.2,-1.2,1.2), cmap="turbo")
    pl.colorbar(img, shrink=0.7)
    pl.scatter(input_data[:,0], input_data[:,1], c=[(['b', 'r'])[int(d>0)] for d in output_data], s=100, edgecolors='black')
    pl.title(f'MLP {idx+1} feature space output')
    pl.show()

In [ ]:

# Note for later: next section will use 3 instead of 1 because 3 classes, and ~ supermax 
layers.Dense(1, activation="tanh"),   # next 
